# Templating with Jinja

Jinja is an external library (you need to `pip install jinja2` to have it available) that is used for creating templates that can be rendered using different input values.

In [2]:
from jinja2 import Template

In [51]:
template = Template('Hello {{ name }}')

template.render({'name':'Ada'})

'Hello Ada'

In [52]:
template.render(name='Alan')

'Hello Alan'

## How this differs from str.format()

Looking at the above we may think that we can just as well use str.format:

In [5]:
str_template = 'Hello {name}'

str_template.format(name='Ada')

'Hello Ada'

In [6]:
str_template.format(name='Alan')

'Hello Alan'

Jinja provides a simple, powerful language for introducing conditionals, iteration, filters and much more into our templates.

In [17]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class ComputerScientist:
    name: str
    contributions: List[str] = field(default_factory=list)
        
ada = ComputerScientist(name='Ada Lovelace', 
                        contributions=['First published algorithm for computer'])

alan = ComputerScientist(name='Alan Kay',
                         contributions=['Designer of Smalltalk', 'Kiddicomp/Dynabook', 'Graphical User Interfaces'])

## Variable Substitution

The most commonly used feature of Jinja is almost certainly plain old variable substitution: you have a variable with some value, and you want that to be inserted to your output.

For this you use double braces: `{{ }}` and put your variable expression within the double braces.

In [55]:
fields_template = Template("Computer Scientist: {{ scientist.name }}\nContributions: {{ scientist.contributions|join('; ') }}")
print(fields_template.render(scientist=alan))

Computer Scientist: Alan Kay
Contributions: Designer of Smalltalk; Kiddicomp/Dynabook; Graphical User Interfaces


## Statements - Conditionals and Iterating

We can also include conditionals and looping constructs, these are expressed using `{% <statement> %}`

In [27]:
fields_template_v2 = Template("""Computer Scientist: {{ scientist.name }}
Contributions:
{% for contribution in scientist.contributions %}
\t{{ contribution }}
{% endfor %}
""")
print(fields_template_v2.render(scientist=alan))

Computer Scientist: Alan Kay
Contributions:

	Designer of Smalltalk

	Kiddicomp/Dynabook

	Graphical User Interfaces



Note that this printing all of our whitespace too. The template string has a newline between "Contributions:" and the for-loop for contributions, so that newline is in our output. [You can control the surrounding whitespace](https://jinja.palletsprojects.com/en/2.11.x/templates/#whitespace-control) more finely using a number of different methods, but this is unlikely to be of interest in our work in Airflow.

In [37]:
fields_template_v3 = Template("""Computer Scientist: {{ scientist.name }}
{% if scientist.contributions %}
Contributions:
{% for contribution in scientist.contributions -%}
{% if not loop.last %}
\t{{ contribution }}
{% else %}
\tand {{ contribution }}
{% endif %}
{%- endfor %}
{% else %}
No contributions!
{% endif %}
""")


print(fields_template_v3.render(scientist=alan))

Computer Scientist: Alan Kay

Contributions:

	Designer of Smalltalk

	Kiddicomp/Dynabook

	and Graphical User Interfaces




In [38]:
print(fields_template_v3.render(scientist=ComputerScientist(name='Patrick')))

Computer Scientist: Patrick

No contributions!



## A practical example

Let's say you want to generate some SQL that Airflow will run every day to UNION ALL `Project Created` events for the previous day and we're going to aggregate it to get unique exporting users (let's ignore our cross platform user id complexities here).

We have three datasets that we need to query: `events, over_android, over_web_events`.

But let's expand the problem space: we need to do this for say five different events and we're expecting to launch on a new platform soon (Blackberry?)

We could just write the SQL for each event and then update the SQL when the new platform launches, or we could create a template.

In [56]:
event_template = Template("""
{% for dataset in datasets %}
SELECT
    timestamp,
    user_id
FROM `over-data.{{ dataset }}.{{ event }}
WHERE _PARTITIONTIME = '{{ ds }}'

{% if not loop.last %}
UNION ALL
{% endif %}
{% endfor %};
""")

In [57]:
datasets = ['events', 'over_android', 'over_web_events', 'over_blackberry']
events = ['project_created', 'project_exported', 'project_opened']
ds = '2021-04-01'

for e in events:
    print(f'=========== {e} ===========')
    print(event_template.render(datasets=datasets, event=e, ds=ds))


=========== project_created ===========


SELECT
    timestamp,
    user_id
FROM `over-data.events.project_created
WHERE _PARTITIONTIME = '2021-04-01'


UNION ALL


SELECT
    timestamp,
    user_id
FROM `over-data.over_android.project_created
WHERE _PARTITIONTIME = '2021-04-01'


UNION ALL


SELECT
    timestamp,
    user_id
FROM `over-data.over_web_events.project_created
WHERE _PARTITIONTIME = '2021-04-01'


UNION ALL


SELECT
    timestamp,
    user_id
FROM `over-data.over_blackberry.project_created
WHERE _PARTITIONTIME = '2021-04-01'


;
=========== project_exported ===========


SELECT
    timestamp,
    user_id
FROM `over-data.events.project_exported
WHERE _PARTITIONTIME = '2021-04-01'


UNION ALL


SELECT
    timestamp,
    user_id
FROM `over-data.over_android.project_exported
WHERE _PARTITIONTIME = '2021-04-01'


UNION ALL


SELECT
    timestamp,
    user_id
FROM `over-data.over_web_events.project_exported
WHERE _PARTITIONTIME = '2021-04-01'


UNION ALL


SELECT
    timestamp,


The whitespace isn't aesthetically pleasing, but this is functional.

## SQL Injection

Whenever we are inserting values into strings that are going to be used in SQL, especially when the values come from a user.

Consider the following scenario:

```
GET /user/search/<name> searches for a user by name
```

Let's see what a naive implementation might look like:

In [45]:
def search_for_user(name):
    '''
    Our search will pretend to do something but actually just return the rendered SQL for us
    '''
    sql_template = Template("SELECT name, profile_picture_url FROM users WHERE name LIKE '{{ name }}%'")
    return sql_template.render(name=name)

In [58]:
search_for_user('al')

"SELECT name, profile_picture_url FROM users WHERE name LIKE 'al%'"

Looks good so far.

What if a malicious user provides the input `al'; DROP TABLE users;`?

In [47]:
search_for_user("al'; DROP TABLE users;")

"SELECT name, profile_picture_url FROM users WHERE name LIKE 'al'; DROP TABLE users;%'"

Uh oh, now we have three SQL statements and one is very destructive.

### Mitigations

Typically SQL Injection is mitigated by using named parameters in SQL driver libraries and setting the input from those parameters, which is then escaped.

The result would be something that would look more like this:

In [49]:
print(search_for_user("al\\'; DROP TABLE users;"))

SELECT name, profile_picture_url FROM users WHERE name LIKE 'al\'; DROP TABLE users;%'
